# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [342]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [343]:
project_name='assignment2' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [345]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [346]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [347]:
your_name = 'Riya_Lamba' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [348]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [349]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
457,57,female,32.01975,0,no,14327.337811
1050,44,female,38.80275,1,no,9707.993894
56,58,female,33.41625,2,no,16464.916187
311,19,female,25.93500,0,no,2102.224960
1288,20,male,41.37000,2,yes,46396.924860


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [350]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [351]:
num_cols = len(dataframe.columns)
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [352]:
input_cols = ['age',	'sex',	'bmi',	'children',	'smoker']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [353]:
categorical_cols = ['sex', 'smoker']
categorical_cols
## mathematically can be done by
'''cols = df.columns

In [67]: num_cols = df._get_numeric_data().columns

In [68]: num_cols
Out[68]: Index([u'0', u'1', u'2'], dtype='object')

In [69]: list(set(cols) - set(num_cols))
Out[69]: ['3', '4']'''

"cols = df.columns\n\nIn [67]: num_cols = df._get_numeric_data().columns\n\nIn [68]: num_cols\nOut[68]: Index([u'0', u'1', u'2'], dtype='object')\n\nIn [69]: list(set(cols) - set(num_cols))\nOut[69]: ['3', '4']"

**Q: What are the column titles of output/target variable(s)?**

In [354]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [ ]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [355]:
!pip install jovian --upgrade -q

In [356]:
import jovian

In [359]:

jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/assignment2


'https://jovian.ai/2019mcb1229/assignment2'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [360]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [361]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[57.     ,  0.     , 32.01975,  0.     ,  0.     ],
        [44.     ,  0.     , 38.80275,  1.     ,  0.     ],
        [58.     ,  0.     , 33.41625,  2.     ,  0.     ],
        ...,
        [59.     ,  0.     , 36.54   ,  2.     ,  0.     ],
        [41.     ,  1.     , 30.24   ,  1.     ,  0.     ],
        [36.     ,  0.     , 31.521  ,  0.     ,  0.     ]]),
 array([[14327.3378105],
        [ 9707.9938945],
        [16464.9161875],
        ...,
        [44661.8357163],
        [ 7601.50435  ],
        [ 6379.332718 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [362]:
inputs = torch.from_numpy(inputs_array)
inputs=inputs.float()
targets = torch.from_numpy(targets_array)
targets= targets.float()

In [363]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [364]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [365]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset,[train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [366]:
batch_size = 40

In [367]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [368]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[40.0000,  1.0000, 37.0650,  3.0000,  0.0000],
        [32.0000,  0.0000, 31.2218,  0.0000,  0.0000],
        [21.0000,  1.0000, 32.8177,  0.0000,  0.0000],
        [37.0000,  1.0000, 29.4263,  2.0000,  0.0000],
        [53.0000,  0.0000, 41.5800,  1.0000,  0.0000],
        [51.0000,  1.0000, 33.9150,  1.0000,  0.0000],
        [23.0000,  1.0000, 44.0055,  0.0000,  0.0000],
        [33.0000,  1.0000, 23.8403,  0.0000,  0.0000],
        [23.0000,  0.0000, 36.7080,  3.0000,  0.0000],
        [64.0000,  0.0000, 35.4900,  1.0000,  1.0000],
        [62.0000,  1.0000, 28.9275,  1.0000,  0.0000],
        [24.0000,  1.0000, 37.6530,  0.0000,  0.0000],
        [54.0000,  0.0000, 37.6058,  3.0000,  0.0000],
        [33.0000,  1.0000, 44.5200,  5.0000,  0.0000],
        [18.0000,  0.0000, 32.9175,  0.0000,  0.0000],
        [42.0000,  0.0000, 26.2342,  2.0000,  0.0000],
        [18.0000,  0.0000, 40.5983,  2.0000,  0.0000],
        [38.0000,  0.0000, 29.2267,  2.0000,  0.0000],
  

Let's save our work by committing to Jovian.

In [369]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/assignment2


'https://jovian.ai/2019mcb1229/assignment2'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [370]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [371]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb.reshape(-1,5))                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets, size_average=None, reduce=None, reduction='mean')                        # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss =  F.l1_loss(out, targets, size_average=None, reduce=None, reduction='mean')                      # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        ##if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
        print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [372]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [373]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1969, -0.3467,  0.4245,  0.3641,  0.1151]], requires_grad=True),
 Parameter containing:
 tensor([-0.3535], requires_grad=True)]

One final commit before we train the model.

In [374]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/assignment2


'https://jovian.ai/2019mcb1229/assignment2'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [375]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [376]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 16219.296875}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [377]:
epochs = 20
lr = 0.01
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1], val_loss: 15490.5312
Epoch [2], val_loss: 14749.7285
Epoch [3], val_loss: 14035.5137
Epoch [4], val_loss: 13335.5967
Epoch [5], val_loss: 12682.4346
Epoch [6], val_loss: 12102.4346
Epoch [7], val_loss: 11575.5332
Epoch [8], val_loss: 11134.9082
Epoch [9], val_loss: 10715.2051
Epoch [10], val_loss: 10350.1240
Epoch [11], val_loss: 10018.2656
Epoch [12], val_loss: 9727.2207
Epoch [13], val_loss: 9495.5605
Epoch [14], val_loss: 9301.7891
Epoch [15], val_loss: 9156.6445
Epoch [16], val_loss: 9034.9629
Epoch [17], val_loss: 8906.3311
Epoch [18], val_loss: 8830.5449
Epoch [19], val_loss: 8764.3340
Epoch [20], val_loss: 8688.8379


In [378]:
epochs = 15
lr = 0.1
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1], val_loss: 8498.4746
Epoch [2], val_loss: 8392.8965
Epoch [3], val_loss: 8366.2129
Epoch [4], val_loss: 8312.4395
Epoch [5], val_loss: 8229.6006
Epoch [6], val_loss: 8235.8145
Epoch [7], val_loss: 8179.1885
Epoch [8], val_loss: 8185.4482
Epoch [9], val_loss: 8175.5493
Epoch [10], val_loss: 8107.7109
Epoch [11], val_loss: 8103.3188
Epoch [12], val_loss: 8090.7720
Epoch [13], val_loss: 8023.0547
Epoch [14], val_loss: 8004.0498
Epoch [15], val_loss: 7992.4946


In [379]:
epochs = 20
lr = 0.15
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1], val_loss: 7990.5859
Epoch [2], val_loss: 8032.4023
Epoch [3], val_loss: 7910.8452
Epoch [4], val_loss: 8055.4844
Epoch [5], val_loss: 7877.5107
Epoch [6], val_loss: 7870.4937
Epoch [7], val_loss: 7856.5483
Epoch [8], val_loss: 7851.4062
Epoch [9], val_loss: 8015.1577
Epoch [10], val_loss: 7926.9399
Epoch [11], val_loss: 7961.3657
Epoch [12], val_loss: 7983.5674
Epoch [13], val_loss: 7830.3682
Epoch [14], val_loss: 7890.8633
Epoch [15], val_loss: 7868.8359
Epoch [16], val_loss: 7823.8672
Epoch [17], val_loss: 7839.2749
Epoch [18], val_loss: 7928.6904
Epoch [19], val_loss: 7827.5625
Epoch [20], val_loss: 7819.6494


In [380]:
epochs = 15
lr = 0.001
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1], val_loss: 7820.0625
Epoch [2], val_loss: 7820.2437
Epoch [3], val_loss: 7820.3701
Epoch [4], val_loss: 7820.5220
Epoch [5], val_loss: 7820.7314
Epoch [6], val_loss: 7820.9907
Epoch [7], val_loss: 7821.1221
Epoch [8], val_loss: 7821.3721
Epoch [9], val_loss: 7821.9907
Epoch [10], val_loss: 7822.1250
Epoch [11], val_loss: 7822.7163
Epoch [12], val_loss: 7822.8657
Epoch [13], val_loss: 7823.5093
Epoch [14], val_loss: 7823.6455
Epoch [15], val_loss: 7824.2490


In [381]:
epochs = 25
lr = 0.01
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [1], val_loss: 7825.2671
Epoch [2], val_loss: 7826.7437
Epoch [3], val_loss: 7827.5405
Epoch [4], val_loss: 7829.6875
Epoch [5], val_loss: 7831.8101
Epoch [6], val_loss: 7838.1460
Epoch [7], val_loss: 7843.3374
Epoch [8], val_loss: 7842.1719
Epoch [9], val_loss: 7841.4858
Epoch [10], val_loss: 7848.0571
Epoch [11], val_loss: 7845.1670
Epoch [12], val_loss: 7842.3413
Epoch [13], val_loss: 7839.6265
Epoch [14], val_loss: 7838.2148
Epoch [15], val_loss: 7843.6016
Epoch [16], val_loss: 7851.7344
Epoch [17], val_loss: 7846.9990
Epoch [18], val_loss: 7852.6758
Epoch [19], val_loss: 7857.8296
Epoch [20], val_loss: 7862.0938
Epoch [21], val_loss: 7866.8311
Epoch [22], val_loss: 7859.7178
Epoch [23], val_loss: 7863.9893
Epoch [24], val_loss: 7856.5444
Epoch [25], val_loss: 7862.1694


**Q: What is the final validation loss of your model?**

In [382]:
val_loss = history1 + history2 + history3 + history4 + history5 

Let's log the final validation loss to Jovian and commit the notebook

In [383]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [384]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/assignment2


'https://jovian.ai/2019mcb1229/assignment2'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [385]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = torch.max(model(inputs), dim=1)               # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [386]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([24.0000,  0.0000, 27.9300,  0.0000,  0.0000])
Target: tensor([3685.7351])
Prediction: tensor([4695.5278])


In [309]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([51.0000,  0.0000, 42.6930,  0.0000,  0.0000])
Target: tensor([11949.5732])
Prediction: tensor([11682.0957])


In [310]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([48.0000,  1.0000, 32.3190,  3.0000,  0.0000])
Target: tensor([12270.7744])
Prediction: tensor([11839.6602])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [311]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/2019mcb1229/02-insurance-linear-regression


'https://jovian.ai/2019mcb1229/02-insurance-linear-regression'